<h5>6.4.2 가중치 감소</h5>

- `가중치 감소(weight decay)`: 학습 과정에서 큰 가중치에 대해서 그에 상응하는 큰 페널티를 부과하여 오버피팅을 억제하는 방법  
- 신경망 학습의 목적: 손실 함수의 값을 줄이는 것

A. `L2 법칙(가중치의 제곱 법칙)`  
- 일반적으로 자주 쓰임  
$${1 \over 2} \lambda W$$
- 모든 가중치 각각의 손실 함수에 더함

- $\mathbf{\lambda}$: 정규화의 세기를 조절하는 하이퍼파라미터  
$\equiv$ 크게 설정할수록 큰 가중치에 대한 페널티 부과  
- ${1 \over 2}$: 미분 결과 $\lambda W$ 를 조절하는 상수

- 실험결과  
    A. 훈련 데이터와 시험 데이터에 대한 정확도 차이가 비교적 감소  
    B. 훈련 데이터에 대한 정확도가 1.0에 도달하지 못함

- `장점`: 간단하게 구현 가능, 어느 정도 지나친 학습 억제 가능
- `단점`: 복잡한 신경망 모델의 오버피팅 억제가 여려울 수 있음

B. L1 법칙: 절댓값의 합  
C. L $\infty$ 법칙(Max 법칙): 각 원소의 절댓값 중 가장 큰 것에 해당

<h5>6.4.3 드롭아웃(dropout)</h5>

A. `훈련 때`  
  - `뉴런을 무작위로 삭제하면서 학습하는 방법`  
  - 삭제된 뉴런은 신호를 전달하지 않음

B. `시험 때`  
  - 모든 뉴런에 신호 전달  
  - 각 뉴런의 출력에 훈련 때 삭제한 비율을 곱하여 출력  
  - 실제로 딥러닝 프레임워크들에서 비율을 곱하지 않기도 함

In [5]:
import re
import numpy as np

class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None                                                # 순전파 때, 삭제할 뉴런을 False로 표시

    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio   # x와 같은 형상의 배열을 무작위로 생성-> 값이 dropout_ratio보다 큰 원소만 True로 설정
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
            return dout * self.mask                                     # ReLU와 동일한 동작 (순전파와 동일하게 작동)

- 실험결과
    - 훈련 데이터와 시험 데이터에 대한 정확도 차이 감소
    - 훈련 데이터에 대한 정확도 1.0에 도달 X
    - 표현력을 높이면서도 오버피팅 억제 가능

- `앙상블 학습(ensemble learning)`
    - 개별적으로 학습시킨 여러 모델의 출력을 평균 내어 추론하는 방식

    - ex) 신경망에서  
        A. 훈련 때, 같거나 비슷한 구조의 네트워크를 n개 준비하여 따로 학습시킨 후,  
        B. 시험 때, n개의 출력을 평균내어 답하는 방식  
    - 신경망의 정확도 몇 %정도 개선한다는 사실 실험적으로 증명

    - 드롭아웃과 밀접  
        $\equiv$ '학습 때, 뉴런을 무작위로 삭제하는 행위를 매번 다른 모델을 학습시키는 것으로 해석한다' 라는 맥락에서 유사  
        $\equiv$ '추론 때, 뉴런의 출력에 삭제한 비율을 곱함으로써 여러 모델을 평균내는 것과 유사한 효과를 얻는다'
        
        $\equiv$ 드롭아웃은 앙상블 학습과 같은 효과를 하나의 네트워크로 구현했다고 판단할 수 있음